In [9]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("C:\\Users\\disha\\Downloads\\newdata.csv")
data = data.set_index('timestamp')

In [6]:
def test_stationarity(data, column):
    result = adfuller(data[column])
    p_value = result[1]
    if p_value < 0.05:
        return 'Stationary'
    else:
        return 'Non-stationary'
    
adf_test_results = {
    col: test_stationarity(data, col)
    for col in data.columns
}

for col, result in adf_test_results.items():
    print(f"{col} is {result}")



metric_1 is Stationary
metric_2 is Stationary
metric_3 is Stationary
metric_4 is Stationary
metric_5 is Stationary
metric_6 is Stationary
metric_7 is Stationary
metric_8 is Stationary
metric_9 is Stationary
metric_10 is Stationary
metric_11 is Stationary
metric_12 is Stationary
metric_13 is Stationary
metric_14 is Stationary
metric_15 is Stationary
metric_16 is Stationary
metric_17 is Stationary
metric_18 is Stationary
metric_19 is Stationary
metric_20 is Stationary
metric_21 is Stationary
metric_22 is Stationary
metric_23 is Stationary
metric_24 is Stationary
metric_25 is Stationary
metric_26 is Stationary
metric_27 is Stationary
metric_28 is Stationary
metric_29 is Stationary
metric_30 is Stationary
Overall anomaly is Stationary
Major anomaly is Stationary


In [5]:
def differencing(data, column, order):
    differenced_data = data[column].diff(order)
    differenced_data.fillna(differenced_data.mean(), inplace=True)
    return differenced_data
for col in data.columns:
    if col=='Overall anomaly' or col=='Major anomaly':
        pass
    else:
        data[col] = differencing(data, col, 1)

In [117]:
data.head

<bound method NDFrame.head of                    metric_1   metric_2    metric_3   metric_4   metric_5  \
timestamp                                                                  
01-01-2021 00:00   1.097634  -1.006654    1.141968  -1.328572  -1.344053   
01-01-2021 01:00  19.855529  -8.730685   19.933339 -28.593807  30.672484   
01-01-2021 02:00 -37.258256  21.848049  -21.250860  16.201519  12.873568   
01-01-2021 03:00   7.384102  17.761899    5.687282  -8.273482 -26.326804   
01-01-2021 04:00  36.986141  50.240251   16.409309  18.963892   4.498941   
...                     ...        ...         ...        ...        ...   
02-11-2021 19:00 -40.700762 -31.238566   89.534914   2.968500  39.206184   
02-11-2021 20:00  48.547450  -9.819345   31.738334  11.527873  -6.716985   
02-11-2021 21:00 -40.122031   6.091424   21.988333  -7.870189 -28.716839   
02-11-2021 22:00   3.676711  -9.906262 -456.528199 -14.878151 -17.325682   
02-11-2021 23:00  31.013056 -19.022465    7.640540 -16.059

In [7]:
from statsmodels.tsa.api import VAR
max_lag = 20
var_model = VAR(data)
# select the best lag order
lag_results = var_model.select_order(max_lag)
selected_lag = lag_results.aic
print(selected_lag) 

C:\Users\disha\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


20


In [10]:
def find_anomalies(squared_errors):
    threshold = np.mean(squared_errors) + np.std(squared_errors)
    predictions = (squared_errors >= threshold).astype(int)
    return predictions, threshold
var = VAR(data)
var_fitresults = var.fit(selected_lag)
squared_errors = var_fitresults.resid.sum(axis=1) ** 2
predictions, threshold = find_anomalies(squared_errors)

C:\Users\disha\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  self._init_dates(dates, freq)


In [11]:
z=1
threshold = np.mean(squared_errors) + z * np.std(squared_errors)

In [13]:
datas = data.iloc[selected_lag:, :]
datas['Predictions'] = predictions.values
datas

C:\Users\disha\AppData\Local\Temp\ipykernel_15388\637843476.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datas['Predictions'] = predictions.values


,metric_1,metric_2,metric_3,metric_4,metric_5,metric_6,metric_7,metric_8,metric_9,metric_10,...,metric_24,metric_25,metric_26,metric_27,metric_28,metric_29,metric_30,Overall anomaly,Major anomaly,Predictions
timestamp,,,,,,,,,,,,,,,,,,,,,
01-01-2021 20:00,-14.488514,-50.300960,42.086833,17.089339,-16.864648,12.595158,-14.407573,53.901714,39.814168,4.448178,...,-36.745320,-431.488771,499.597228,19.232144,13.754282,48.938374,-91.005009,0,0,1
01-01-2021 21:00,73.364366,65.958252,-16.304977,-9.835087,31.560978,5.901906,-0.884961,-1.826564,-99.579306,15.331441,...,44.232070,356.487918,14.615957,9.131368,-21.079722,-629.412054,64.594502,1,0,0
01-01-2021 22:00,-311.067964,-52.210871,-529.452123,1.747161,-41.204308,275.457534,13.942154,-13.844965,89.429049,-8.838864,...,-11.371084,64.604507,0.175702,-24.126907,3.058169,524.188353,-14.755995,1,1,0
01-01-2021 23:00,512.343178,29.398457,505.069543,9.582064,42.197608,-558.617091,2.750843,4.726949,-33.266227,-940.732117,...,-16.475325,-63.882867,10.371826,1.706609,10.809605,-0.540487,-10.018044,0,0,0
01-02-2021 00:00,-279.795816,-9.862101,-55.025566,-49.853514,1.513019,284.980704,-21.262785,-42.168673,-18.258015,868.080186,...,12.973552,21.145523,-37.456519,30.748573,-36.832778,27.252483,-14.517617,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
02-11-2021 19:00,-72.388394,-53.129313,65.999412,16.403132,104.929911,-13.121220,5.667052,56.440748,9.201160,5.781428,...,62.794552,-12.427065,-568.457001,117.233136,40.654122,33.324938,492.037819,0,0,0
02-11-2021 20:00,89.248212,21.419221,-57.796580,8.559373,-45.923169,-28.287713,6.765558,-0.375522,-28.015476,-18.201084,...,-71.983449,-431.852865,509.570793,-84.337775,-4.556797,19.796440,34.936616,1,0,0
02-11-2021 21:00,-88.669481,15.910769,-9.750001,-19.398062,-21.999854,10.212786,-48.967557,-38.854517,62.413391,29.267335,...,10.145299,486.732372,-10.891272,24.415887,-18.428584,-628.176341,2.544088,0,0,0


In [122]:
datas.to_csv('C:\\Users\\disha\\Downloads\\vardataset3.csv')

In [16]:
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score
datas=pd.read_csv('C:\\Users\\disha\\Downloads\\vardataset3.csv', parse_dates=['timestamp'])

In [18]:
#VAR
true_values = datas['Major anomaly'].values
predicted_values = datas['Predictions'].values
precision = precision_score(true_values, predicted_values)
f1=f1_score(true_values, predicted_values)
recall=recall_score(true_values, predicted_values)
accuracy=accuracy_score(true_values, predicted_values)
print("precision= ",precision)
print("F1 score= ",f1)
print("recall= ",recall)
print("accuracy= ",accuracy)

precision=  0.1015625
F1 score=  0.15950920245398775
recall=  0.37142857142857144
accuracy=  0.861336032388664


In [14]:
anomalous_count = len(datas[datas['Predictions'] == 1])
print("Number of anomalous tuples:", anomalous_count)

Number of anomalous tuples: 107
